In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
%matplotlib inline

In [2]:
# load dataset
data = pd.read_csv('./data/cardekho_imputated.csv')
data.head()
data.isnull().sum()
data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15411 entries, 0 to 15410
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         15411 non-null  int64  
 1   car_name           15411 non-null  object 
 2   brand              15411 non-null  object 
 3   model              15411 non-null  object 
 4   vehicle_age        15411 non-null  int64  
 5   km_driven          15411 non-null  int64  
 6   seller_type        15411 non-null  object 
 7   fuel_type          15411 non-null  object 
 8   transmission_type  15411 non-null  object 
 9   mileage            15411 non-null  float64
 10  engine             15411 non-null  int64  
 11  max_power          15411 non-null  float64
 12  seats              15411 non-null  int64  
 13  selling_price      15411 non-null  int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 1.6+ MB


In [3]:
data.head()

,Unnamed: 0,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [27]:
# data.drop('Unnamed: 0', axis=1, inplace=True)
data.drop('car_name', axis=1, inplace=True)
data.drop('brand', axis=1, inplace=True)

In [25]:
# data.head()
# data['model'].unique()
# data['model'].value_counts()

In [6]:
# get all the different type of columns
num_features = [feature for feature in data.columns if data[feature].dtype != 'object']
print(num_features)
cat_features = [feature for feature in data.columns if data[feature].dtype == 'object']
print(cat_features)

['vehicle_age', 'km_driven', 'mileage', 'engine', 'max_power', 'seats', 'selling_price']
['car_name', 'brand', 'model', 'seller_type', 'fuel_type', 'transmission_type']


# Feature Encoding here

In [22]:
from sklearn.preprocessing import LabelEncoder

# we don't user one hot encode cause it make extra 120 columns for 120 unique values
encoder = LabelEncoder()
data['model'] = encoder.fit_transform(data['model'])

In [68]:
data.head()

,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,7,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,54,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,118,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,7,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,38,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


In [29]:

X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [9]:
cat_features
data['seller_type'].unique(), data['fuel_type'].unique(), data['transmission_type'].unique()

(array(['Individual', 'Dealer', 'Trustmark Dealer'], dtype=object),
 array(['Petrol', 'Diesel', 'CNG', 'LPG', 'Electric'], dtype=object),
 array(['Manual', 'Automatic'], dtype=object))

In [31]:
num_features = X.select_dtypes(exclude='object').columns
onehot_columns = ['seller_type', 'fuel_type', 'transmission_type']

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transforamtion = StandardScaler()
oh_tranformation = OneHotEncoder(drop='first')

transformer = ColumnTransformer(
    [
        ("OneHotEncoder", oh_tranformation, onehot_columns),
        ("StandardScaler", num_transforamtion, num_features),
    ], remainder='passthrough'  # remainder='passthrough' means other features not chnages or delete without we need
)

In [32]:
X = transformer.fit_transform(X)

In [33]:
X

array([[ 1.        ,  0.        ,  0.        , ..., -1.32425883,
        -1.26335238, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.43257082, -0.40302241],
       [ 1.        ,  0.        ,  0.        , ..., -0.55471774,
        -0.47911321, -0.40302241],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.02291783,
         0.06822523, -0.40302241],
       [ 0.        ,  0.        ,  1.        , ...,  1.32979434,
         0.91715831,  2.07344426],
       [ 0.        ,  0.        ,  0.        , ...,  0.02099878,
         0.39588361, -0.40302241]], shape=(15411, 14))

# Train Test Split

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size= 0.2,
    random_state=42
)

# Model selection and training the Model

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [44]:
def evalute(true, prediction):
    mae = mean_absolute_error(true, prediction)
    mse = mean_squared_error(true, prediction)
    rmse = np.sqrt(mse)
    r2_score_acc = r2_score(true, prediction)
    
    return mae, mse, rmse, r2_score_acc

In [45]:
# Beggining the model
models= {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbors Regressor" : KNeighborsRegressor(),
    "Decision Tree Regressor" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # evaluete the performance
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2_score = evalute(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2_score = evalute(y_test, y_test_pred)
    
    print(f"-------------model name: {name}--------------")
    print(f"Train - MAE: {model_train_mae}, MSE: {model_train_mse}, RMSE: {model_train_rmse}, R2: {model_train_r2_score}")
    print(f"Test  - MAE: {model_test_mae}, MSE: {model_test_mse}, RMSE: {model_test_rmse}, R2: {model_test_r2_score}\n")
    
    print('------' * 20)
    

-------------model name: Linear Regression--------------
Train - MAE: 268101.6070829937, MSE: 306756099359.7596, RMSE: 553855.6665411663, R2: 0.6217719576765959
Test  - MAE: 279618.57941584283, MSE: 252550062888.5656, RMSE: 502543.5930230985, R2: 0.6645109298852004

------------------------------------------------------------------------------------------------------------------------
-------------model name: Lasso--------------
Train - MAE: 268099.2226498115, MSE: 306756104248.3742, RMSE: 553855.6709544231, R2: 0.6217719516489697
Test  - MAE: 279614.7461034126, MSE: 252549134806.78134, RMSE: 502542.66963789385, R2: 0.6645121627547996

------------------------------------------------------------------------------------------------------------------------
-------------model name: Ridge--------------
Train - MAE: 268059.8014688311, MSE: 306756818740.9266, RMSE: 553856.3159709624, R2: 0.6217710706848424
Test  - MAE: 279557.2168930274, MSE: 252540243247.96863, RMSE: 502533.8229890289, R2: 

# Hyperparameter Tuning

In [48]:
knn_params ={
    "n_neighbors" : [2, 3, 10, 20, 40, 50]
}

rf_params = {
    "max_depth": [5, 8, 15, None, 10],
    "max_features": [5, 7, 8, "sqrt", "log2"],
    "n_estimators": [100, 200, 500, 1000],
    "min_samples_split": [2, 5, 10, 20],
    # "min_samples_leaf": [1, 2, 4],
    # "bootstrap": [True, False],
}

In [49]:
randomcv_models = [
    ("KNN", KNeighborsRegressor(), knn_params),
    ("RF", RandomForestRegressor(), rf_params)
]

In [50]:
from sklearn.model_selection import RandomizedSearchCV

model_params = {}

for name, model, param in randomcv_models:
    random = RandomizedSearchCV(
        estimator=model,
        param_distributions=param,
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=-1
    )
    
    random.fit(X_train, y_train)
    model_params[name] = random.best_params_
    
    
for model_name in model_params:
    print(f"-----------{model_name}------------")
    print(model_params[model_name])

e:\ML-Bootcamp-Practical\venv\lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=100. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 6 candidates, totalling 18 fits
Fitting 3 folds for each of 100 candidates, totalling 300 fits
-----------KNN------------
{'n_neighbors': 10}
-----------RF------------
{'n_estimators': 100, 'min_samples_split': 2, 'max_features': 7, 'max_depth': 15}


In [51]:
# Beggining the model
models= {
    "K-Neighbors Regressor" : KNeighborsRegressor(
        n_neighbors=10
    ),
    "Random Forest Regressor" : RandomForestRegressor(
        n_estimators=100,
        min_samples_split=2,
        max_features=7,
        max_depth=15
    ),
}

for name, model in models.items():
    model.fit(X_train, y_train)
    
    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # evaluete the performance
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2_score = evalute(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2_score = evalute(y_test, y_test_pred)
    
    print(f"-------------model name: {name}--------------")
    print(f"Train - MAE: {model_train_mae}, MSE: {model_train_mse}, RMSE: {model_train_rmse}, R2: {model_train_r2_score}")
    print(f"Test  - MAE: {model_test_mae}, MSE: {model_test_mse}, RMSE: {model_test_rmse}, R2: {model_test_r2_score}\n")
    
    print('------' * 20)
    

-------------model name: K-Neighbors Regressor--------------
Train - MAE: 103472.04737183647, MSE: 132103731766.70993, RMSE: 363460.7706021517, R2: 0.8371170582947722
Test  - MAE: 117496.21310411936, MSE: 69636909424.46481, RMSE: 263888.06230003055, R2: 0.9074938975612508

------------------------------------------------------------------------------------------------------------------------
-------------model name: Random Forest Regressor--------------
Train - MAE: 54747.28882544065, MSE: 17374256988.926353, RMSE: 131811.444832861, R2: 0.9785776673342086
Test  - MAE: 98418.97738741773, MSE: 45399768437.56312, RMSE: 213072.21413774983, R2: 0.9396906659917729

------------------------------------------------------------------------------------------------------------------------


In [52]:
import joblib

joblib.dump(models["Random Forest Regressor"], "random_forest_car_price_model.pkl")

['random_forest_car_price_model.pkl']

In [69]:
import streamlit as st

# Streamlit UI
st.title("🚗 Car Price Prediction App")
st.write("Enter car details to predict its price.")

# User Input Fields
model_input = st.text_input("Car Model (e.g., Grand, Alto, Swift)")

vehicle_age = st.number_input("Vehicle Age (in years)", min_value=0, max_value=30, step=1)
km_driven = st.number_input("Kilometers Driven", min_value=0, step=1000)
seller_type = st.selectbox("Seller Type", ["Individual", "Dealer"])
fuel_type = st.selectbox("Fuel Type", ["Petrol", "Diesel", "CNG"])
transmission_type = st.selectbox("Transmission", ["Manual", "Automatic"])
mileage = st.number_input("Mileage (km/l)", min_value=0.0, step=0.1)
engine = st.number_input("Engine Capacity (cc)", min_value=500, step=100)
max_power = st.number_input("Max Power (HP)", min_value=10, step=1)
seats = st.number_input("Number of Seats", min_value=2, max_value=10, step=1)

2025-02-05 12:56:49.044 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.843 
  command:

    streamlit run e:\ML-Bootcamp-Practical\venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-05 12:56:49.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.846 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.848 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.849 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.850 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-05 12:56:49.850 Thread 'MainThread